In [40]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random
import time
import re
from datetime import datetime

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [46]:
Ciudad="Roma"

In [49]:
driver=webdriver.Chrome()
url = "https://www.hellotickets.es/"
    # Abrimos navegador y maximizamos página en la url
driver.get(url)
driver.maximize_window()
sleep(3)
# Aceptar cookies
try:
    driver.find_element("css selector","#__layout > div > div.cookie-banner.cookie-banner-desktop.cookie-banner--v2 > div.cookie-banner__btns-container--v2 > button:nth-child(2)").click()
except:    
    print('No encuentro el boton cookies')
sleep(2)
# Escribir nombre de la ciudad
try:
    driver.find_element("css selector","#input-search").send_keys({Ciudad}, Keys.ENTER)
except:    
    print('No he podido buscar')
sleep(2)
# Pulsar en pagina
try:
    driver.find_element("css selector","#__layout > div > main > div > div > div > div.section-search > div > div.input-search__results-container > div.input-search__cities-container.input-search__cities-container--top > ul > li > a").click()
except:    
    print('No encuentro ciudad')
# Aceptar cookies
sleep(2)
try:
    driver.find_element("css selector","#__layout > div > div.cookie-banner.cookie-banner-desktop > div.cookie-banner__btns-container > button:nth-child(1)").click()
except:    
    print('No encuentro el boton Cookies')
# Scroll en la pagina
start_time = time.time()
while time.time() - start_time < 40:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Pequeña pausa para que la página cargue más contenido dinámico

# Obtén el HTML de la página ya cargada
html = driver.page_source

# Usa BeautifulSoup para procesar el HTML
soup = BeautifulSoup(html, 'html.parser')

listaactividades = soup.find_all('a', class_='product-grid__wrapper')

fechascrap = datetime.now()

print('Scrap finish')


No encuentro el boton Cookies
Scrap finish


In [50]:
datos_actividades = []

for actividad in listaactividades:
    # Extraer el link
    link = actividad.get('href', '')
    
    # Extraer el nombre de la actividad
    nombre_actividad = actividad.find('span', class_='product-grid__title')
    nombre_actividad = nombre_actividad.get_text(strip=True) if nombre_actividad else ''
    
    # Extraer el precio desde
    precio = actividad.find('div', class_='about-activity-grid-price__new-price')
    if precio:
        precio = precio.get_text(strip=True)
        # Utilizamos una expresión regular para extraer solo los números
        precio = re.findall(r'\d+', precio)
        precio = precio[0] if precio else ''  # Si hay números, tomamos el primero, si no, dejamos vacío
    else:
        precio = ''

    # Confirmación inmediata
    confirmacion_inmediata = ''
    condiciones = actividad.find_all('span', class_='product-grid__conditions-item')
    for condicion in condiciones:
        if 'Confirmación inmediata' in condicion.get_text():
            confirmacion_inmediata = 'Si'
            break
    
    # Cancelación gratuita
    cancelacion_gratuita = actividad.find('span', class_='about-activity-grid-cancellation__text')
    cancelacion_gratuita = cancelacion_gratuita.get_text(strip=True) if cancelacion_gratuita else 'No'

       # Extraer el número de valoración
    valoracion = actividad.find('span', class_='rating-stars__rating')
    valoracion = valoracion.get_text(strip=True) if valoracion else ''
    
    # Extraer el número de opiniones (siempre están entre paréntesis)
    opiniones = actividad.find('span', class_='rating-stars__review-count-text')
    if opiniones:
        opiniones = re.findall(r'\d+', opiniones.get_text(strip=True))
        opiniones = opiniones[0] if opiniones else ''  # Tomar el primer número encontrado
    else:
        opiniones = ''

    # Agregar los datos extraídos a la lista
    datos_actividades.append({
        'Actividad': nombre_actividad,
        'Precio (Por persona)': precio,
        'Confirmacion Inmediata': confirmacion_inmediata,
        'Cancelacion Gratuita': cancelacion_gratuita,
        'Valoracion': valoracion,
        'Nº Opiniones': opiniones,
        'Link': link,
        'Fecha consulta':fechascrap
    })

# Crear el DataFrame a partir de la lista de diccionarios
df_actividades = pd.DataFrame(datos_actividades)

df_actividades


,Actividad,Precio (Por persona),Confirmacion Inmediata,Cancelacion Gratuita,Valoracion,Nº Opiniones,Link,Fecha consulta
0,"Visita guiada a los Museos Vaticanos, Capilla ...",105,Si,Cancelación gratuita,4.8,3706,https://www.hellotickets.es/italia/roma/visita...,2024-10-19 21:07:03.018222
1,Visita guiada de la Basílica de San Pedro y la...,52,,Cancelación gratuita,,,https://www.hellotickets.es/italia/roma/visita...,2024-10-19 21:07:03.018222
2,Oferta: Entradas al Coliseo y al Foro Romano +...,90,,No,4.4,2784,https://www.hellotickets.es/italia/roma/entrad...,2024-10-19 21:07:03.018222
3,"Visita guiada del Coliseo, el Foro Romano y el...",42,,Cancelación gratuita,4.7,3799,https://www.hellotickets.es/italia/roma/visita...,2024-10-19 21:07:03.018222
4,Roma Pass y Omnia Card,149,,Cancelación gratuita,4.5,948,https://www.hellotickets.es/italia/roma/roma-p...,2024-10-19 21:07:03.018222
...,...,...,...,...,...,...,...,...
356,Tour nocturno de Roma en Vespa,176,,Cancelación gratuita,4.7,485,https://www.hellotickets.es/italia/roma/tour-n...,2024-10-19 21:07:03.018222
357,Entradas sin colas para el Castillo de Sant'An...,25,Si,No,,,https://www.hellotickets.es/italia/roma/entrad...,2024-10-19 21:07:03.018222
358,Entradas sin colas para el Panteón de Roma,13,Si,Cancelación gratuita,,,https://www.hellotickets.es/italia/roma/entrad...,2024-10-19 21:07:03.018222
359,Entrada reservada para el Coliseo,32,,No,,,https://www.hellotickets.es/italia/roma/entrad...,2024-10-19 21:07:03.018222


In [51]:
# Exportación del DataFrame a un archivo CSV
df_actividades.to_csv(f'../data/actividades_en_{Ciudad}.csv', index=False)